In [1]:
import json
import cv2
import numpy as np
import os
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

# Load annotations from JSON file
try:
    with open('annotations.json', 'r') as f:
        annotations = json.load(f)
except Exception as e:
    print("Error loading JSON:", e)
    annotations = {}

# Preprocess annotations to match the [xmin, ymin, xmax, ymax] format
image_annotations = {}
for image_name, coords in annotations.items():
    try:
        box = coords
        image_annotations[image_name] = [box]
    except Exception as e:
        print(f"Error processing annotations for {image_name}: {e}")

class CustomDataGenerator(Sequence):
    def __init__(self, image_paths, mask_paths, annotations, batch_size, image_size):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.annotations = annotations
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size: (index + 1) * self.batch_size]
        batch_mask_paths = self.mask_paths[index * self.batch_size: (index + 1) * self.batch_size]

        # Print filenames for debugging
        for img_path in batch_image_paths:
            print("Image Path:", img_path)
        
        # Updated code to access annotations using filenames
        batch_annotations = []
        for img_path in batch_image_paths:
            filename = os.path.basename(img_path)
            print("Processing:", filename)
            annotation = self.annotations.get(filename)
            if annotation is not None:
                batch_annotations.append(annotation)
            else:
                print("Annotation not found for:", filename)

        print("Batch annotations:", batch_annotations)

        images = []
        masks = []

        # Load and preprocess images and masks
        for img_path, mask_path in zip(batch_image_paths, batch_mask_paths):
            image = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            image = cv2.resize(image, self.image_size)
            mask = cv2.resize(mask, self.image_size)

            images.append(image)
            masks.append(mask)

        # Print the shapes of images, masks, and annotations
        images_batch = np.array(images)
        masks_batch = np.array(masks)
        annotations_batch = np.array(batch_annotations)
        print("Image batch shape:", images_batch.shape)
        print("Mask batch shape:", masks_batch.shape)
        print("Annotation batch shape:", annotations_batch.shape)

        return images_batch, masks_batch

In [2]:

# Define the Mean Squared Error (MSE) loss function
mse_loss = MeanSquaredError()

# Update these paths to the actual directories containing your images and masks
image_directory = "images"
mask_directory = "masks"

# List all files in the image and mask directories
image_filenames = os.listdir(image_directory)
mask_filenames = os.listdir(mask_directory)

# Create full file paths by joining directory paths with filenames
image_paths = [os.path.join(image_directory, filename) for filename in image_filenames]
mask_paths = [os.path.join(mask_directory, filename) for filename in mask_filenames]

# Rest of your code remains the same
annotations = image_annotations  # Extracted annotations

batch_size = 1
image_size = (224, 224)  # Adjust according to your needs
num_classes = 1  # Number of object classes

data_generator = CustomDataGenerator(image_paths, mask_paths, annotations, batch_size, image_size)

# Build the model
def build_object_detection_model(image_size, num_classes):
    backbone = ResNet50(include_top=False, input_shape=image_size, weights='imagenet')
    for layer in backbone.layers:
        layer.trainable = False
    x = backbone.output
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    output_layer = Dense(4)(x)  # Predict [xmin, ymin, xmax, ymax]
    model = Model(inputs=backbone.input, outputs=output_layer)
    return model

model = build_object_detection_model(image_size + (3,), num_classes)

# Compile the model using the Mean Squared Error (MSE) loss function
model.compile(optimizer=Adam(), loss=mse_loss)

# Train the model
model.fit(data_generator, epochs=50)

FileNotFoundError: [WinError 3] System nie może odnaleźć określonej ścieżki: 'images'